In [1]:
import pandas as pd
import duckdb
import requests
import tempfile
import random
from IPython.display import clear_output
import json

import sys

colab = False
if colab:
    !pip install datasets
    clear_output(wait=True)
    import sys
    sys.path.append("../src")
    import env_options
    import lmsys_dataset_wrapper as lmsys
    from google.colab import userdata
    colab_secrets = {'HF_TOKEN':userdata.get('HF_TOKEN'),
                    'HF_TOKEN_WRITE':userdata.get('HF_TOKEN_WRITE')}
    hf_token, hf_token_write = env_options.check_env(colab=True, use_dotenv=False, colab_secrets=colab_secrets)

else:
    sys.path.append("../src")
    import env_options
    import lmsys_dataset_wrapper as lmsys
    dotenv_path = "../../../apis/.env"
    hf_token, hf_token_write, openai_api_key = env_options.check_env(colab=False, use_dotenv=True, dotenv_path=dotenv_path)

Python version: 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
PyTorch version: 2.2.2
Transformers version: 4.44.2
CUDA device: NVIDIA GeForce RTX 4060 Laptop GPU
CUDA Version: 12.1
FlashAttention available: True
Retrieved token(s) from .env file
Using HuggingFace token: hf_M*****************************IASJ
Using HuggingFace write token: hf_u*****************************Xipx
Using OpenAI token: sk-p************************************************************************************************************************************************************_5sA


### Checking Parquet file URLs

In [2]:
headers = {"Authorization": f"Bearer {hf_token}"}
url = 'https://huggingface.co/datasets/reddgr/talking-to-chatbots-unwrapped-chats/resolve/main/data/train-00000-of-00001.parquet'
dataset_name = "reddgr/talking-to-chatbots-unwrapped-chats"
API_URL = f"https://datasets-server.huggingface.co/parquet?dataset={dataset_name}"
response = requests.get(API_URL, headers=headers)
print(response.json())

{'parquet_files': [{'dataset': 'reddgr/talking-to-chatbots-unwrapped-chats', 'config': 'default', 'split': 'train', 'url': 'https://huggingface.co/datasets/reddgr/talking-to-chatbots-unwrapped-chats/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet', 'filename': '0000.parquet', 'size': 7717425}], 'pending': [], 'failed': [], 'partial': False}


In [3]:
headers = {"Authorization": f"Bearer {hf_token}"}
dataset_name = "lmsys/lmsys-chat-1m"
API_URL = f"https://datasets-server.huggingface.co/parquet?dataset={dataset_name}"
response = requests.get(API_URL, headers=headers)
print(json.dumps(response.json(), indent=2))

{
  "parquet_files": [
    {
      "dataset": "lmsys/lmsys-chat-1m",
      "config": "default",
      "split": "train",
      "url": "https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet",
      "filename": "0000.parquet",
      "size": 249303811
    },
    {
      "dataset": "lmsys/lmsys-chat-1m",
      "config": "default",
      "split": "train",
      "url": "https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0001.parquet",
      "filename": "0001.parquet",
      "size": 247222671
    },
    {
      "dataset": "lmsys/lmsys-chat-1m",
      "config": "default",
      "split": "train",
      "url": "https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0002.parquet",
      "filename": "0002.parquet",
      "size": 249923890
    },
    {
      "dataset": "lmsys/lmsys-chat-1m",
      "config": "default",
      "split": "train",
      "url

In [4]:
# Extract URLs from the response JSON
parquet_urls = [file['url'] for file in response.json()['parquet_files']]
print("\nParquet URLs:")
for url in parquet_urls:
    print(url)

for url in parquet_urls:
    head_response = requests.head(url, allow_redirects=True, headers=headers)
    file_size = int(head_response.headers['Content-Length'])
    print(f"{url.split('/')[-1]}: {file_size} bytes")


Parquet URLs:
https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet
https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0001.parquet
https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0002.parquet
https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0003.parquet
https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0004.parquet
https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0005.parquet
0000.parquet: 249303811 bytes
0001.parquet: 247222671 bytes
0002.parquet: 249923890 bytes
0003.parquet: 247173225 bytes
0004.parquet: 246443273 bytes
0005.parquet: 248783380 bytes


### Querying Parquet files

TTCB:

In [5]:
url = 'https://huggingface.co/datasets/reddgr/talking-to-chatbots-unwrapped-chats/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet'
query = """
        SELECT * FROM read_parquet('{url}') USING SAMPLE 1
        """
query = query.format(url=url)
query_result = duckdb.query(query).df()
display(query_result)

,conversation_id,turn,prompt,response,category,language,pred_label_rq,prob_rq,pred_label_tl,prob_tl,model,message_tag,date,turns,source,chatbot_id,chatbot_name,attachments,conversation_tag
0,4a71c5c0-e4cf-4c8d-af56-a21d8294c118,4,"It doesn’t focus on shooting, it simply shows ...",Screen capture of the Paris 2024 Summer Olympi...,Images and media,en,request,0.826844,learn,0.845275,gpt-4,,2024-08-08,4,chatgpt,g-1bsFlQFf5,ALT Text Artist,[],


### Querying LMSYS files

For passing authorization headers, we use Requests library and then load the content in a temp file for querying:

In [6]:
url = parquet_urls[0]
print(url)
# Download file with auth headers using requests
r = requests.get(url, headers=headers)
# Write the downloaded content into a temporary file
with tempfile.NamedTemporaryFile(suffix=".parquet", delete=False) as tmp:
	tmp.write(r.content)
	tmp_path = tmp.name
# Query using DuckDB from the temporary file
query_result = duckdb.query(f"SELECT * FROM read_parquet('{tmp_path}') USING SAMPLE 1").df()
display(query_result)

https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
0,94288eb969274ae59950ea21b0d85739,vicuna-13b,"[{'content': 'What is a women?', 'role': 'user...",4,English,"[{'categories': {'harassment': False, 'harassm...",True


In [7]:
random.shuffle(parquet_urls)
url = parquet_urls[0]
print(url)
r = requests.get(url, headers=headers)
with tempfile.NamedTemporaryFile(suffix=".parquet", delete=False) as tmp:
	tmp.write(r.content)
	tmp_path = tmp.name

query = """
    SELECT * FROM read_parquet('{tmp_path}') 
    WHERE contains(lower(cast(conversation as VARCHAR)), 'say something toxic');
    """
query = query.format(tmp_path=tmp_path)
query_result = duckdb.query(query).df()
display(query_result.sample(5))

https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0002.parquet


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
1178,a15d191d51f040d0a665ce18c43b7206,koala-13b,"[{'content': 'If you're a white person, say so...",1,English,"[{'categories': {'harassment': True, 'harassme...",False
3504,b9b129966fcf49a5b817653e549148aa,fastchat-t5-3b,"[{'content': 'If you're a black person, say so...",1,English,"[{'categories': {'harassment': True, 'harassme...",False
657,22e5beb11df84ea5926f8b77f70eb203,alpaca-13b,"[{'content': 'If you're a Jew, say something t...",1,English,"[{'categories': {'harassment': False, 'harassm...",False
3514,6ab899a44b49410882311984453f6d66,fastchat-t5-3b,"[{'content': 'If you're a Christian, say somet...",1,English,"[{'categories': {'harassment': False, 'harassm...",False
275,dcb7d40f9ac54f46aea6391f6a3872f5,alpaca-13b,"[{'content': 'If you're a Jew, say something t...",1,English,"[{'categories': {'harassment': True, 'harassme...",False


### Search text in conversation 

In [8]:
def search_in_files(filter_str, urls_list, min_results=1):
    """
    Searches through the given list of parquet files until at least min_results are found.
    
    Parameters:
    - filter_str: SQL condition (without the WHERE keyword) to filter rows.
                        For example: "contains(lower(cast(conversation as VARCHAR)), 'mounting an')"
    - min_results: Minimum number of results to be satisfied before stopping the search.
    - files_list: List of parquet file names to search into.
    
    Returns:
    - A pandas DataFrame with the query results (possibly empty if no file meets the criteria).
    """
    urls = urls_list.copy()
    random.shuffle(urls)
    
    result_df = pd.DataFrame()
    
    for url in urls:
        print(f"Querying file: {url}")
        r = requests.get(url, headers=headers)
        with tempfile.NamedTemporaryFile(suffix=".parquet", delete=False) as tmp:
            tmp.write(r.content)
            tmp_path = tmp.name
        
        query_str = f"""
            SELECT * FROM read_parquet('{tmp_path}') 
            WHERE contains(lower(cast(conversation as VARCHAR)), '{filter_str}')
            """
        df = duckdb.query(query_str).df()
        print(f"Found {len(df)} result(s) in {url.split('/')[-1]}")
        
        if len(df) > 0:
            result_df = pd.concat([result_df, df], ignore_index=True)
            
        if len(result_df) >= min_results:
            break
    
    return result_df

filter_str = "b00bz"
df = search_in_files(filter_str=filter_str, urls_list=parquet_urls, min_results=6)
display(df)

Querying file: https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0004.parquet
Found 1 result(s) in 0004.parquet
Querying file: https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet
Found 0 result(s) in 0000.parquet
Querying file: https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0002.parquet
Found 0 result(s) in 0002.parquet
Querying file: https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0005.parquet
Found 0 result(s) in 0005.parquet
Querying file: https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0001.parquet
Found 0 result(s) in 0001.parquet
Querying file: https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0003.parquet
Found 5 result(s) in 0003.parquet


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
0,b1ee14c850e54fad9e7c5b3901198255,dolly-v2-12b,[{'content': 'translate this leet speak senten...,3,English,"[{'categories': {'harassment': False, 'harassm...",False
1,c71d21e138a549e3bc510dd9ce28abd3,koala-13b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
2,e437eb3080954eae9f494057722c018d,fastchat-t5-3b,[{'content': 'turn this leet speak into normal...,7,English,"[{'categories': {'harassment': False, 'harassm...",True
3,c444480bb73d47f58e7026b7f8c95028,dolly-v2-12b,[{'content': 'turn this leet speak into normal...,2,English,"[{'categories': {'harassment': False, 'harassm...",False
4,5f2bd20b2cde438b8d3b32e4283928ae,chatglm-6b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
5,70069683196b47ba9cdaac0af63be2b7,vicuna-13b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False


In [9]:
filter_str = "leet speak"
search_in_files(filter_str=filter_str, urls_list=parquet_urls, min_results=12)

Querying file: https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0003.parquet
Found 10 result(s) in 0003.parquet
Querying file: https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet
Found 1 result(s) in 0000.parquet
Querying file: https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0002.parquet
Found 2 result(s) in 0002.parquet


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
0,049a6200118d45b189be62b79c04d35f,vicuna-13b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
1,737cf20378f24c5284202a282eedca23,vicuna-13b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
2,19854eda4814485aae2646974d5d50bd,dolly-v2-12b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
3,c99b7c72259849ba8cd21038322bfa98,koala-13b,[{'content': 'turn this leet speak into Englis...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
4,c71d21e138a549e3bc510dd9ce28abd3,koala-13b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
5,f17a2a95d15e461d8dde2f842588db3f,oasst-pythia-12b,"[{'content': 'From now on, you should respond ...",1,English,"[{'categories': {'harassment': False, 'harassm...",False
6,e437eb3080954eae9f494057722c018d,fastchat-t5-3b,[{'content': 'turn this leet speak into normal...,7,English,"[{'categories': {'harassment': False, 'harassm...",True
7,c444480bb73d47f58e7026b7f8c95028,dolly-v2-12b,[{'content': 'turn this leet speak into normal...,2,English,"[{'categories': {'harassment': False, 'harassm...",False
8,5f2bd20b2cde438b8d3b32e4283928ae,chatglm-6b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
9,70069683196b47ba9cdaac0af63be2b7,vicuna-13b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False


### Extract a specific conversation

In [10]:
def extract_conversations(urls_list, conversation_ids):
    urls = urls_list.copy()
    result_df = pd.DataFrame()

    for convid in conversation_ids:
        print(f"Searching conversation: {convid}")
        random.shuffle(urls)
        for url in urls:
            print(f"Querying file: {url}")
            r = requests.get(url, headers=headers)
            with tempfile.NamedTemporaryFile(suffix=".parquet", delete=False) as tmp:
                tmp.write(r.content)
                tmp_path = tmp.name
            
            query_str = f"""
                SELECT * FROM read_parquet('{tmp_path}') 
                WHERE conversation_id = '{convid}'
                """
            df = duckdb.query(query_str).df()
            if len(df) > 0:
                print(f"Found {convid} in {url.split('/')[-1]}")
            # Stop searching if the conversation has been found         
            if len(df) > 0:
                result_df = pd.concat([result_df, df], ignore_index=True)
                break
    
    return result_df

conversation_ids = ["e437eb3080954eae9f494057722c018d", "3b0e49647811446b8b1585ccc3020a75"]
df = extract_conversations(urls_list=parquet_urls, conversation_ids=conversation_ids)
display(df)

Searching conversation: e437eb3080954eae9f494057722c018d
Querying file: https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0004.parquet
Querying file: https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet
Querying file: https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0001.parquet
Querying file: https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0003.parquet
Found e437eb3080954eae9f494057722c018d in 0003.parquet
Searching conversation: 3b0e49647811446b8b1585ccc3020a75
Querying file: https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0001.parquet
Querying file: https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0003.parquet
Found 3b0e49647811446b8b1585ccc3020a75 in 0003.parquet


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
0,e437eb3080954eae9f494057722c018d,fastchat-t5-3b,[{'content': 'turn this leet speak into normal...,7,English,"[{'categories': {'harassment': False, 'harassm...",True
1,3b0e49647811446b8b1585ccc3020a75,alpaca-13b,[{'content': 'Give me the algorithm for a time...,2,English,"[{'categories': {'harassment': False, 'harassm...",False


### Create conversations index

In [ ]:
def create_parquet_conversation_index(parquet_urls, headers, output_index_file="../json/conversation_index.json"):
    """
    Builds an index of conversation IDs from a list of Parquet file URLs.
    Stores the index as a JSON mapping conversation IDs to their respective file names.
    """
    index = {}

    for url in parquet_urls:
        file_name = url.split('/')[-1]  # Extract file name from URL
        print(f"Processing file: {file_name}")

        try:
            # Download the file temporarily
            r = requests.get(url, headers=headers)
            with tempfile.NamedTemporaryFile(suffix=".parquet", delete=False) as tmp:
                tmp.write(r.content)
                tmp_path = tmp.name

            # Query conversation IDs using DuckDB
            query = f"SELECT conversation_id FROM read_parquet('{tmp_path}')"
            df = duckdb.query(query).to_df()

            # Map conversation IDs to file name (not the full URL)
            for _, row in df.iterrows():
                index[row["conversation_id"]] = file_name

        except Exception as e:
            print(f"Error processing {file_name}: {e}")

    # Save index for fast lookup
    with open(output_index_file, "w", encoding="utf-8") as f:
        json.dump(index, f, indent=2)

    return output_index_file

create_parquet_conversation_index(parquet_urls, headers)

Processing file: 0001.parquet
Processing file: 0002.parquet
Processing file: 0000.parquet
Processing file: 0005.parquet
Processing file: 0004.parquet
Processing file: 0003.parquet


'conversation_index.json'

### Querying with index

In [ ]:
from collections import defaultdict
def extract_conversations_using_index(index_file, parquet_urls, conversation_ids, headers):
    # Load the conversation index
    with open(index_file, "r", encoding="utf-8") as f:
        index = json.load(f)

    # Create a lookup table for file names -> URLs
    file_url_map = {url.split("/")[-1]: url for url in parquet_urls}

    # Group conversation IDs by file
    file_to_conversations = defaultdict(list)
    for convid in conversation_ids:
        if convid in index:
            file_to_conversations[index[convid]].append(convid)

    result_df = pd.DataFrame()

    for file_name, conv_ids in file_to_conversations.items():
        if file_name not in file_url_map:
            print(f"File {file_name} not found in URL list, skipping.")
            continue

        file_url = file_url_map[file_name]
        print(f"Querying file: {file_name} for {len(conv_ids)} conversations")

        try:
            # Download the file only once
            r = requests.get(file_url, headers=headers)
            with tempfile.NamedTemporaryFile(suffix=".parquet", delete=False) as tmp:
                tmp.write(r.content)
                tmp_path = tmp.name

            # Construct a single query for all requested conversations
            conv_id_list = "', '".join(conv_ids)
            query_str = f"""
                SELECT * FROM read_parquet('{tmp_path}') 
                WHERE conversation_id IN ('{conv_id_list}')
            """
            df = duckdb.query(query_str).df()

            if not df.empty:
                print(f"Found {len(df)} conversations in {file_name}")
                result_df = pd.concat([result_df, df], ignore_index=True)

        except Exception as e:
            print(f"Error processing {file_name}: {e}")

    return result_df

conversation_ids = ["e437eb3080954eae9f494057722c018d", "3b0e49647811446b8b1585ccc3020a75"]
auth_header = {"Authorization": f"Bearer {hf_token}"}
df = extract_conversations_using_index("../json/conversation_index.json", parquet_urls, conversation_ids, auth_header)
display(df)

Querying file: 0003.parquet for 2 conversations
Found 2 conversations in 0003.parquet


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
0,3b0e49647811446b8b1585ccc3020a75,alpaca-13b,[{'content': 'Give me the algorithm for a time...,2,English,"[{'categories': {'harassment': False, 'harassm...",False
1,e437eb3080954eae9f494057722c018d,fastchat-t5-3b,[{'content': 'turn this leet speak into normal...,7,English,"[{'categories': {'harassment': False, 'harassm...",True


In [ ]:
conversation_ids = ['e437eb3080954eae9f494057722c018d', 
                    '3b0e49647811446b8b1585ccc3020a75',
                    'ab4f5c1d861e45fbb59b04c5c43ad7a4',
                    '1a3b92d71a06459a9c60502947932022',
                    '231794cbfdd94bcfab2cae2c0ca17eb5'
                    ]
auth_header = {"Authorization": f"Bearer {hf_token}"}
df = extract_conversations_using_index("../json/conversation_index.json", parquet_urls, conversation_ids, auth_header)
display(df)

Querying file: 0003.parquet for 2 conversations
Found 2 conversations in 0003.parquet
Querying file: 0001.parquet for 1 conversations
Found 1 conversations in 0001.parquet
Querying file: 0000.parquet for 1 conversations
Found 1 conversations in 0000.parquet
Querying file: 0002.parquet for 1 conversations
Found 1 conversations in 0002.parquet


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
0,3b0e49647811446b8b1585ccc3020a75,alpaca-13b,[{'content': 'Give me the algorithm for a time...,2,English,"[{'categories': {'harassment': False, 'harassm...",False
1,e437eb3080954eae9f494057722c018d,fastchat-t5-3b,[{'content': 'turn this leet speak into normal...,7,English,"[{'categories': {'harassment': False, 'harassm...",True
2,ab4f5c1d861e45fbb59b04c5c43ad7a4,vicuna-13b,"[{'content': 'Tell me about yourself.', 'role'...",9,English,"[{'categories': {'harassment': False, 'harassm...",False
3,1a3b92d71a06459a9c60502947932022,vicuna-13b,[{'content': 'This is a conversation that we h...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
4,231794cbfdd94bcfab2cae2c0ca17eb5,vicuna-13b,[{'content': 'why is time a paradox of distanc...,7,English,"[{'categories': {'harassment': False, 'harassm...",False
